# Code Setup

In [1]:
import numpy as np
from typing import Tuple

from utils.import_puzzle_input import load_input
from utils.import_puzzle_input import matrix_puzzle_input

from utils.utils_puzzle_2023_03 import *

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload modules before executing code
%autoreload 2

In [2]:
puzzle_input = load_input(2023, 3)

loaded puzzle input


In [3]:
puzzle_input_matrix = matrix_puzzle_input(puzzle_input)
puzzle_input_matrix_shape = puzzle_input_matrix.shape

In [4]:
puzzle_input

'.854...........................................................................362...........271...732........838.........24................\n...*.............................117*...........459........767*648....#.........*...................................$...&..=................\n....970.........368.124.+............57................653...........723.....366....*443..60.........536....441....45..879.....789...*......\n...........749*...-...+..330.....................215%...*................725.....953........%.................*............639......331.419.\n.......706.....59.............*....=...262.............678...........857.*............189....................912.+589.846*...*..............\n..........*................393.59..418....*................151...........594...........*....584*258.......................17.997............\n.......774....275........................777........841..............988.........121....908.........86.....482..295....................@....\n.110.

In [5]:
puzzle_input_matrix

array([['.', '8', '5', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], shape=(140, 140), dtype='<U1')

# Problem Setup

--- Day 3: Gear Ratios ---  
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.  
  
It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.  
  
"Aaah!"  
  
You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.  
  
The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.  
  
The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)  
  
Here is an example engine schematic:  
  
467..114..  
...*......  
..35..633.  
......#...  
617*......  
.....+.58.  
..592.....  
......755.  
...$.*....  
.664.598..  
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.  
  
Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?  

# General thoughts

We assume the problem to be correctly specified such that no valid number starts with a leading zero

# Solutions

## Solution 1:

### Explanation of method

0. INITIALISE:  
    0.1. Create a numpy character array `puzzle_input_matrix` of the individual characters of the puzzle.  
    0.2. Initialise arrays for row and column indices of the, to be determined, valid numbers of the `puzzle_input_matrix`.  
1. FIND SYMBOLS:  
    1.1. Find the non-number, non-`.`-symbols.  
2. GET ENTRYPOINT NUMBERS:  
    2.1. For each `symbol_entry` entry of `puzzle_input_matrix` that is found to be a non-number, non-`.`-symbol  
        2.1.1. Retrieve the eight neighbouring entries  
        2.1.2. Record the indices of those neighbouring entries that are numbers  
3. GET ALL VALID NUMBERS:  
    3.1. For each row, create a set of sorted tuples. Each tuple will contain the column indices for uninterupted numbers.  
    3.2. To get a sorted tuple of column indices of uninterupted numbers, we union a 'right' and 'left' tuple.  
    3.3. To get the `uninterupted_index_right` tuple, iterate over the valid numbers for that particular row. For a valid number, add the next column index to the tuple, if the entry is a number.  
    3.4. To get the `uninterupted_index_left` tuple, iterate over the valid numbers for that particular row. For a valid number, add the previous column index to the tuple, if the entry is a number.  
    3.5. Take the sort the union of the left and right indices, and add it to the set of uninterupted for that row. The set structure will remove duplicates of tuples.  
4. ADD ALL VALID NUMBERS:
    4.1. Combine together the numbers behind the indices found in each tuple, and add the number to a sum.

### Implementation

#### Step 1:

In [6]:
is_digit_mask = is_digit(puzzle_input_matrix)
is_not_period_mask = puzzle_input_matrix != "."

In [7]:
enabler_symbol_mask = is_not_period_mask * ~is_digit_mask
enabler_symbol_indices = np.where(enabler_symbol_mask)

#### Step 2:

In [8]:
neighbour_indices = get_surrounding_indices(enabler_symbol_indices, puzzle_input_matrix_shape)
neighbour_entries = puzzle_input_matrix[neighbour_indices]

In [9]:
neighbour_mask = get_mask_from_indices(neighbour_indices, puzzle_input_matrix.shape)

In [10]:
digit_neighbour_indices = tuple(map(lambda x: x[is_digit(neighbour_entries)], neighbour_indices))
# digit_neighbour_indices

#### Step 3

In [11]:
tuple_dict = {key: set() for key in digit_neighbour_indices[0]}
# tuple_dict

In [12]:
for row, col in zip(digit_neighbour_indices[0], digit_neighbour_indices[1]):
    # print(f"row: {row}, col: {col}")
    tuple_dict[row].add(get_uninterupted_indices_of_numbers(puzzle_input_matrix, row, col))

In [15]:
# get_tuple_dict(puzzle_input_matrix, digit_neighbour_indices)

#### Step 4:

In [17]:
list_of_valid_numbers = get_valid_list_of_numbers_of_puzzle(puzzle_input_matrix, tuple_dict)

In [18]:
sum(list_of_valid_numbers)

553825

### Checking runtime

#### Benchmarking

#### Runtime profiling

## Solution 2:

## Experimenting

In [50]:
def l1_pythagoras(i_1:int, j_1:int, i_2:int, j_2:int) -> int:
    return ((np.abs(i_1 - i_2) <= 1) + (np.abs(j_1 - j_2) <= 1)) >= 1

In [51]:
test_case = np.array([
    list("467..114."),
    list("...*....."),
    list("..35..633"),
    list("......#.."),
    list("617*....."),
    list(".....+.58"),
    list("..592...."),
    list("......755"),
    list("...$.*..."),
    list(".664.598."),
])
test_case

array([['4', '6', '7', '.', '.', '1', '1', '4', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.']], dtype='<U1')

#### Step 1:

In [52]:
test_case_is_digit_mask = is_digit(test_case)
test_case_is_not_period_mask = test_case != "."

In [53]:
test_case_enabler_symbol_mask = test_case_is_not_period_mask * ~test_case_is_digit_mask
test_case_enabler_symbol_indices = np.where(test_case_enabler_symbol_mask)

In [54]:
np.where(test_case_enabler_symbol_mask, test_case, 'Ø')

array([['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', '*', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', '#', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', '*', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', '+', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', '$', 'Ø', '*', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø']], dtype='<U1')

In [55]:
test_case_enabler_symbol_indices = np.where(test_case_enabler_symbol_mask)
test_case_enabler_symbol_indices

(array([1, 3, 4, 5, 8, 8]), array([3, 6, 3, 5, 3, 5]))

In [56]:
def get_surrounding_points(np_where_indices: tuple, matrix_shape) -> tuple:
    
    # Get indices
    row_indices, column_indices = np_where_indices
    
    # set offsets for the eight surrounding entries
    row_offsets = np.array([-1, -1, -1, 0, 0, 1, 1, 1])
    col_offsets = np.array([-1, 0, 1, -1, 1, -1, 0, 1])

    # calculate the neighbour entries
    neighbour_rows = row_indices[:, None] + row_offsets
    neighbour_cols = column_indices[:, None] + col_offsets

    # check that we don't check invalid indices
    proper_index_mask = (neighbour_rows >= 0) & (neighbour_rows < matrix_shape[0]) & \
                        (neighbour_cols >= 0) & (neighbour_cols < matrix_shape[1])
    
    # Get the well defined neighbour indices
    neighbour_rows = neighbour_rows[proper_index_mask]
    neighbour_cols = neighbour_cols[proper_index_mask]

    return (neighbour_rows, neighbour_cols)

#### START INTERLUDE: TESTING `get_surrounding_points_old`

In [57]:
def get_surrounding_points_old(np_where_indices: tuple, matrix_shape) -> tuple:
    
    # Get indices
    row_indices, column_indices = np_where_indices
    
    # set offsets for the eight surrounding entries
    row_offsets = np.array([-1, -1, -1, 0, 0, 1, 1, 1])
    col_offsets = np.array([-1, -1, -1, 0, 0, 1, 1, 1])

    # calculate the neighbour entries
    neighbour_rows = row_indices[:, None] + row_offsets
    neighbour_cols = column_indices[:, None] + col_offsets

    # check that we don't check invalid indices
    proper_index_mask = (neighbour_rows >= 0) & (neighbour_rows < matrix_shape[0]) & \
                        (neighbour_cols >= 0) & (neighbour_cols < matrix_shape[1])
    
    # Get the well defined neighbour indices
    neighbour_rows = neighbour_rows[proper_index_mask]
    neighbour_cols = neighbour_cols[proper_index_mask]

    return (neighbour_rows, neighbour_cols)

Let's test `get_surrounding_points`

In [58]:
test_array = np.array([
    ['1','2','3'],
    ['4','b','6'],
    ['7','8','9'],
    ])

In [59]:
test_array

array([['1', '2', '3'],
       ['4', 'b', '6'],
       ['7', '8', '9']], dtype='<U1')

In [60]:
np.where(test_array == "b")

(array([1]), array([1]))

In [61]:
test_array_neighbour_indices = get_surrounding_points_old(np.where(test_array == "b"), test_array.shape)
test_array_neighbour_indices

(array([0, 0, 0, 1, 1, 2, 2, 2]), array([0, 0, 0, 1, 1, 2, 2, 2]))

In [62]:
test_array_neighbour_mask = np.zeros(test_array.shape, dtype = bool) 
test_array_neighbour_mask[test_array_neighbour_indices] = True

In [63]:
test_array_neighbour_mask

array([[ True, False, False],
       [False,  True, False],
       [False, False,  True]])

#### FINISH INTERLUDE: TESTING `get_surrounding_points_old`

NOTE: From python 3.9+ you may use `tuple` instead of `typing.Tuple` for type hinting.

In [64]:
test_case_neighbour_indices = get_surrounding_indices(test_case_enabler_symbol_indices, test_case.shape)
test_case_neighbour_entries = test_case[test_case_neighbour_indices]
test_case_neighbour_entries

array(['7', '.', '.', '.', '.', '3', '5', '.', '.', '6', '3', '.', '.',
       '.', '.', '.', '.', '.', '.', '7', '.', '.', '.', '.', '.', '.',
       '.', '.', '.', '2', '.', '.', '.', '.', '.', '.', '.', '6', '4',
       '.', '.', '.', '7', '.', '.', '.', '5', '9'], dtype='<U1')

In [65]:
is_digit(test_case_neighbour_entries)

array([ True, False, False, False, False,  True,  True, False, False,
        True,  True, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False,  True,  True, False, False, False,  True, False, False,
       False,  True,  True])

In [66]:
np.where(is_digit(test_case_neighbour_entries))

(array([ 0,  5,  6,  9, 10, 19, 29, 37, 38, 42, 46, 47]),)

In [67]:
test_case_neighbour_mask = get_mask_from_indices(test_case_neighbour_indices, test_case.shape)
test_case_neighbour_mask

array([[False, False,  True,  True,  True, False, False, False, False],
       [False, False,  True, False,  True, False, False, False, False],
       [False, False,  True,  True,  True,  True,  True,  True, False],
       [False, False,  True,  True,  True,  True, False,  True, False],
       [False, False,  True, False,  True,  True,  True,  True, False],
       [False, False,  True,  True,  True, False,  True, False, False],
       [False, False, False, False,  True,  True,  True, False, False],
       [False, False,  True,  True,  True,  True,  True, False, False],
       [False, False,  True, False,  True, False,  True, False, False],
       [False, False,  True,  True,  True,  True,  True, False, False]])

In [68]:
test_case_neighbour_masked = np.where(test_case_neighbour_mask, test_case, 'Ø')
test_case_neighbour_masked

array([['Ø', 'Ø', '7', '.', '.', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', '.', 'Ø', '.', 'Ø', 'Ø', 'Ø', 'Ø'],
       ['Ø', 'Ø', '3', '5', '.', '.', '6', '3', 'Ø'],
       ['Ø', 'Ø', '.', '.', '.', '.', 'Ø', '.', 'Ø'],
       ['Ø', 'Ø', '7', 'Ø', '.', '.', '.', '.', 'Ø'],
       ['Ø', 'Ø', '.', '.', '.', 'Ø', '.', 'Ø', 'Ø'],
       ['Ø', 'Ø', 'Ø', 'Ø', '2', '.', '.', 'Ø', 'Ø'],
       ['Ø', 'Ø', '.', '.', '.', '.', '7', 'Ø', 'Ø'],
       ['Ø', 'Ø', '.', 'Ø', '.', 'Ø', '.', 'Ø', 'Ø'],
       ['Ø', 'Ø', '6', '4', '.', '5', '9', 'Ø', 'Ø']], dtype='<U1')

In [69]:
test_case_neighbour_masked_is_digit = np.where(is_digit(test_case_neighbour_masked))
test_case_neighbour_masked_is_digit

(array([0, 2, 2, 2, 2, 4, 6, 7, 9, 9, 9, 9]),
 array([2, 2, 3, 6, 7, 2, 4, 6, 2, 3, 5, 6]))

##### Redone solution:

In [70]:
test_case_neighbour_entries

array(['7', '.', '.', '.', '.', '3', '5', '.', '.', '6', '3', '.', '.',
       '.', '.', '.', '.', '.', '.', '7', '.', '.', '.', '.', '.', '.',
       '.', '.', '.', '2', '.', '.', '.', '.', '.', '.', '.', '6', '4',
       '.', '.', '.', '7', '.', '.', '.', '5', '9'], dtype='<U1')

In [71]:
is_digit(test_case_neighbour_entries)

array([ True, False, False, False, False,  True,  True, False, False,
        True,  True, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False,  True,  True, False, False, False,  True, False, False,
       False,  True,  True])

In [72]:
test_case_neighbour_indices

(array([0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 3, 3, 3, 4, 4, 5,
        5, 5, 4, 4, 4, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 9, 7, 7, 7, 8,
        8, 9, 9, 9]),
 array([2, 3, 4, 2, 4, 2, 3, 4, 5, 6, 7, 5, 7, 5, 6, 7, 2, 3, 4, 2, 4, 2,
        3, 4, 4, 5, 6, 4, 6, 4, 5, 6, 2, 3, 4, 2, 4, 2, 3, 4, 4, 5, 6, 4,
        6, 4, 5, 6]))

In [74]:
tuple(map(lambda x: x[is_digit(test_case_neighbour_entries)], test_case_neighbour_indices))

(array([0, 2, 2, 2, 2, 4, 6, 9, 9, 7, 9, 9]),
 array([2, 2, 3, 6, 7, 2, 4, 2, 3, 6, 5, 6]))

In [75]:
test_case_neighbour_masked_is_digit

(array([0, 2, 2, 2, 2, 4, 6, 7, 9, 9, 9, 9]),
 array([2, 2, 3, 6, 7, 2, 4, 6, 2, 3, 5, 6]))

##### investigate the difference

In [ ]:
neighbour_masked_is_digit_redone_set = set(zip(neighbour_masked_is_digit_redone[0], neighbour_masked_is_digit_redone[1]))

In [ ]:
len(neighbour_masked_is_digit_redone_set)

1774

In [ ]:
neighbour_masked_is_digit_set = set(zip(neighbour_masked_is_digit[0], neighbour_masked_is_digit[1]))

In [ ]:
neighbour_masked_is_digit_redone_set == neighbour_masked_is_digit_set

True

#### Step 3:

In [31]:
tuple_dict = dict.fromkeys(test_case_neighbour_masked_is_digit[0])
tuple_dict

{np.int64(0): None,
 np.int64(2): None,
 np.int64(4): None,
 np.int64(6): None,
 np.int64(7): None,
 np.int64(9): None}

In [32]:
if is_digit(test_case[0,0]):
    print("yay")

yay


In [33]:
np.array([], dtype = np.int64)

array([], dtype=int64)

In [34]:
np.array([3,2,1], dtype = np.int64)

array([3, 2, 1])

In [35]:
sorted(np.array([3,2,1], dtype = np.int64))

[np.int64(1), np.int64(2), np.int64(3)]

In [36]:
set([3,2,1]) | set([6,5,4])

{1, 2, 3, 4, 5, 6}

In [37]:
a_set = set()

In [38]:
a_set.add("a")

In [39]:
a_set.add("b")

In [40]:
a_set

{'a', 'b'}

In [ ]:
def get_uninterupted_indices_of_numbers(matrix, row, col) -> tuple:
    matrix_shape = matrix.shape
    uninterupted_index_right = set()
    uninterupted_index_left = set()

    if not((0 <= row and row < matrix_shape[0]) and (0 <= col and col < matrix_shape[1])):
        raise IndexError
    
    if not is_digit(matrix[row, col]):
        raise TypeError("the entry for the given row, col pair is not a digit")
    
    next_entry = col
    while is_digit(matrix[row, next_entry]):
        uninterupted_index_right.add(next_entry)
        if next_entry < matrix_shape[1] - 1:
            next_entry += 1
        else:
            break

    previous_entry = col
    while is_digit(matrix[row, previous_entry]):
        uninterupted_index_left.add(previous_entry)
        if previous_entry >= 0 + 1:
            previous_entry -= 1
        else:
            break

    uninterupted_index = tuple(sorted((uninterupted_index_left | uninterupted_index_right)))
    return uninterupted_index


In [48]:
test_case

array([['4', '6', '7', '.', '.', '1', '1', '4', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.']], dtype='<U1')

In [49]:
get_uninterupted_indices_of_numbers(test_case, 0, 0)

(0, 1, 2)

In [50]:
get_uninterupted_indices_of_numbers(test_case, 0, 7)

(5, 6, 7)

In [51]:
test_case[0, (5,6,7)]

array(['1', '1', '4'], dtype='<U1')

Let's create our dict of sets containing index tuples

In [57]:
tuple_dict = {key: set() for key in test_case_neighbour_masked_is_digit[0]}
tuple_dict

{np.int64(0): set(),
 np.int64(2): set(),
 np.int64(4): set(),
 np.int64(6): set(),
 np.int64(7): set(),
 np.int64(9): set()}

In [58]:
for row, col in zip(test_case_neighbour_masked_is_digit[0], test_case_neighbour_masked_is_digit[1]):
    print(f"row: {row}, col: {col}")
    tuple_dict[row].add(get_uninterupted_indices_of_numbers(test_case, row, col))

row: 0, col: 2
row: 2, col: 2
row: 2, col: 3
row: 2, col: 6
row: 2, col: 7
row: 4, col: 2
row: 6, col: 4
row: 7, col: 6
row: 9, col: 2
row: 9, col: 3
row: 9, col: 5
row: 9, col: 6


In [59]:
tuple_dict

{np.int64(0): {(np.int64(0), np.int64(1), np.int64(2))},
 np.int64(2): {(np.int64(2), np.int64(3)),
  (np.int64(6), np.int64(7), np.int64(8))},
 np.int64(4): {(np.int64(0), np.int64(1), np.int64(2))},
 np.int64(6): {(np.int64(2), np.int64(3), np.int64(4))},
 np.int64(7): {(np.int64(6), np.int64(7), np.int64(8))},
 np.int64(9): {(np.int64(1), np.int64(2), np.int64(3)),
  (np.int64(5), np.int64(6), np.int64(7))}}

In [62]:
def get_number_from_text_array(text_numbers: np.ndarray) -> int:
    number = sum(int(digit) * (10 ** i) for i, digit in enumerate(reversed(text_numbers)))
    return number

In [63]:
get_number_from_text_array(test_case[0, (5,6,7)])

114

In [ ]:
def get_sum_of_puzzle(tuple_dict: dict, matrix: np.ndarray):
    val = 0 
    for row in tuple_dict:
        for tuple_val in tuple_dict[row]:
            val += get_number_from_text_array(matrix[row, tuple_val])
            print(get_number_from_text_array(matrix[row, tuple_val]))
    return val

In [80]:
get_sum_of_puzzle(tuple_dict, test_case)

4361

In [68]:
test_case

array([['4', '6', '7', '.', '.', '1', '1', '4', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.']], dtype='<U1')

In [81]:
def get_valid_list_of_numbers_of_puzzle(tuple_dict: dict, matrix: np.ndarray) -> int:
    valid_list_of_numbers = np.array([get_number_from_text_array(matrix[row, tuple_val]) for row in tuple_dict for tuple_val in tuple_dict[row]])
    return valid_list_of_numbers

In [82]:
get_valid_list_of_numbers_of_puzzle(tuple_dict, test_case)

array([467,  35, 633, 617, 592, 755, 598, 664])

In [78]:
def get_sum_of_puzzle_2(tuple_dict: dict, matrix: np.ndarray) -> int:
    listy_list = [get_number_from_text_array(matrix[row, tuple_val]) for row in tuple_dict for tuple_val in tuple_dict[row]]
    print(listy_list)

In [79]:
get_sum_of_puzzle_2(tuple_dict, test_case)

[467, 35, 633, 617, 592, 755, 598, 664]


In [72]:
def get_sum_of_puzzle_3(tuple_dict: dict, matrix: np.ndarray):
    val = sum(get_number_from_text_array(matrix[row, tuple_val]) for row in tuple_dict for tuple_val in tuple_dict[row])
    return val

In [73]:
get_sum_of_puzzle_3(tuple_dict, test_case)

4361